In [1]:
%autosave 0

Autosave disabled


In [2]:
%load_ext autoreload
%autoreload 2
%aimport train
%aimport mcts

In [3]:
from typing import Optional
from games.connect4 import Connect4
from games.game import Game
from injectors import get_mcts, get_network, get_replay_buffer, get_trainer
from train import self_play
import playing

In [4]:
import cProfile
import pstats

profiling = True

In [5]:
# device = "cpu"
device = "cuda"

network = get_network(Connect4)

replay_buffer = get_replay_buffer(Connect4)
mcts = get_mcts(network)

trainer = get_trainer(
    device,
    network,
    replay_buffer,
)

game = Connect4()


def mcts_policy(game: Game):
    return mcts.search(game)

In [6]:
%%time
pr: Optional[cProfile.Profile] = None
if profiling:
    with cProfile.Profile() as pr:
        self_play(Connect4, mcts, replay_buffer, num_games=1)
else:
    self_play(Connect4, mcts, replay_buffer, num_games=1)

Games played:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 11.2 s, sys: 101 ms, total: 11.3 s
Wall time: 11.5 s


In [7]:
if pr is not None:
    stats = pstats.Stats(pr)
    stats.sort_stats("cumtime").print_stats(30)
    pr.dump_stats("self_play.prof")

         8228473 function calls (8211359 primitive calls) in 11.028 seconds

   Ordered by: cumulative time
   List reduced from 698 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     6551    0.014    0.000    5.600    0.001 /home/piotrek/miniconda3/envs/mpum-big-project/lib/python3.12/site-packages/torch/nn/modules/module.py:1732(_wrapped_call_impl)
     6551    5.578    0.001    5.585    0.001 /home/piotrek/miniconda3/envs/mpum-big-project/lib/python3.12/site-packages/torch/nn/modules/module.py:1740(_call_impl)
     8812    0.016    0.000    2.837    0.000 /home/piotrek/Studia/2-rok/4-sem/MPUM/Project/AlphaZero/games/connect4.py:85(is_terminal)
     8800    0.004    0.000    2.835    0.000 /home/piotrek/Studia/2-rok/4-sem/MPUM/Project/AlphaZero/mcts.py:120(is_terminal)
    11073    0.004    0.000    2.799    0.000 /home/piotrek/Studia/2-rok/4-sem/MPUM/Project/AlphaZero/games/connect4.py:80(reward)
     6629    1.041    0.000   

In [ ]:
trainer.train(batch_size=1)

In [ ]:
playing.play_game(game, mcts_policy_fn=mcts_policy)

In [ ]:
playing.play_game(game, mcts_policy_fn=mcts_policy)